<a href="https://colab.research.google.com/github/kkrusere/youTube-comments-Analyzer/blob/main/LLM_fine-tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

#mounting google drive
drive.mount('/content/drive')

########################################

#changing the working directory
os.chdir("/content/drive/MyDrive/EV NLP Data")

!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/EV NLP Data


In [2]:
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !kaggle datasets download -d widhiwinata/twitter-and-reddit-sentiment-analysis
# !unzip twitter-and-reddit-sentiment-analysis.zip
# !rm twitter-and-reddit-sentiment-analysis.zip

In [3]:
!pip install nltk

We are going to do a semi supervised lable propergation

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
sentimentdata = pd.read_csv("sentimentdataset.csv")
sentimentdata.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [6]:
sentimentdata = sentimentdata[['Text', 'Sentiment']]
sentimentdata.head()

,Text,Sentiment
0,Enjoying a beautiful day at the park! ...,Positive
1,Traffic was terrible this morning. ...,Negative
2,Just finished an amazing workout! 💪 ...,Positive
3,Excited about the upcoming weekend getaway! ...,Positive
4,Trying out a new recipe for dinner tonight. ...,Neutral


In [7]:
# Cleaning-up
sentimentdata['Sentiment'] = sentimentdata['Sentiment'].str.strip()
sentimentdata['Sentiment'] = sentimentdata['Sentiment'].str.lower()


In [8]:
sentimentdata.shape

(732, 2)

In [9]:
sentimentdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       732 non-null    object
 1   Sentiment  732 non-null    object
dtypes: object(2)
memory usage: 11.6+ KB


In [11]:
sentimentdata.head()

,Text,Sentiment
0,Enjoying a beautiful day at the park! ...,positive
1,Traffic was terrible this morning. ...,negative
2,Just finished an amazing workout! 💪 ...,positive
3,Excited about the upcoming weekend getaway! ...,positive
4,Trying out a new recipe for dinner tonight. ...,neutral


In [12]:
sentimentdata['Sentiment'].value_counts()

Sentiment
positive          45
joy               44
excitement        37
contentment       19
neutral           18
                  ..
lostlove           1
emotionalstorm     1
suffering          1
bittersweet        1
intrigue           1
Name: count, Length: 191, dtype: int64

In [19]:
def classify_sentiment(sentiment):
    positive_words = ['positive', 'happiness', 'joy', 'love', 'amusement', 'enjoyment', 'admiration', 'affection', 'awe', 'elation', 'euphoria', 'contentment', 'serenity', 'gratitude', 'hope', 'empowerment', 'compassion', 'tenderness', 'arousal', 'enthusiasm', 'fulfillment', 'reverence', 'playful', 'free-spirited', 'inspired', 'confident', 'zest', 'hopeful', 'proud', 'grateful', 'empathetic', 'compassionate', 'inspiration', 'motivation', 'satisfaction', 'blessed', 'optimism', 'enchantment', 'intrigue', 'playfuljoy', 'mindfulness', 'dreamchaser', 'whimsy', 'harmony', 'creativity', 'radiance', 'wonder', 'rejuvenation', 'adventure', 'awe', 'melodic', 'festivejoy', 'innerjourney', 'freedom', 'dazzle', 'harmony', 'artisticburst', 'radiance', 'wonder', 'inspiration', 'culinaryodyssey', 'curiosity', 'resilience', 'immersion', 'nostalgia', 'spark', 'gratitude', 'marvel', 'serenity', 'joy', 'happiness', 'enthusiasm', 'gratitude', 'positivity', 'kindness', 'friendship', 'love', 'success', 'thrill', 'reflection', 'enchantment', 'exploration', 'awe', 'amazement', 'romance', 'captivation', 'wonder', 'tranquility', 'grandeur', 'emotion', 'energy', 'celebration', 'charm', 'ecstasy', 'hope', 'creativity', 'colorful', 'pride', 'hypnotic', 'connection', 'iconic', 'euphoria', 'journey', 'engagement', 'touched', 'suspense', 'satisfaction', 'admiration', 'triumph', 'heartwarming', 'solace', 'breakthrough', 'joy in baking', 'imagination', 'vibrancy', 'mesmerizing', 'culinary adventure', 'winter magic', 'thrilling journey', 'nature\'s beauty', 'celestial wonder', 'creative inspiration', 'runway creativity', 'ocean\'s freedom', 'whispers of the past', 'positive', 'mischievous', 'happy']

    negative_words = ['negative', 'anger', 'fear', 'sadness', 'disgust', 'disappointed', 'despair', 'loneliness', 'jealousy', 'resentment', 'frustration', 'boredom', 'anxiety', 'intimidation', 'helplessness', 'envy', 'regret', 'bitter', 'confusion', 'overwhelmed', 'jealous', 'devastated', 'envious', 'dismissive', 'shame', 'elation', 'despair', 'grief', 'loneliness', 'frustration', 'anxiety', 'intimidation', 'helplessness', 'jealousy', 'curiosity', 'indifference', 'confusion', 'numbness', 'melancholy', 'nostalgia', 'ambivalence', 'acceptance', 'determination', 'numbness', 'bitterness', 'fearful', 'apprehensive', 'overwhelmed', 'jealous', 'devastated', 'frustrated', 'envious', 'dismissive', 'betrayal', 'suffering', 'emotionalstorm', 'isolation', 'disappointment', 'lostlove', 'melancholy', 'exhaustion', 'sorrow', 'darkness', 'desperation', 'ruins', 'desolation', 'regret', 'grief', 'heartbreak', 'betrayal', 'resilience', 'sorrow', 'loss', 'heartache', 'solitude', 'sad', 'hate', 'bad', 'disgust', 'embarrassed', 'mischievous']

    if sentiment in positive_words:
        return 'Positive'
    elif sentiment in negative_words:
        return 'Negative'
    else:
        return 'Neutral'



In [20]:
# We are using the `classify_sentiment` function to categorize the comment into 3 Positive, Negative and Neutral
sentimentdata['Sentiment'] = sentimentdata['Sentiment'].apply(classify_sentiment)

In [21]:
sentimentdata.head()

,Text,Sentiment
0,Enjoying a beautiful day at the park! ...,Positive
1,Traffic was terrible this morning. ...,Negative
2,Just finished an amazing workout! 💪 ...,Positive
3,Excited about the upcoming weekend getaway! ...,Positive
4,Trying out a new recipe for dinner tonight. ...,Neutral


In [22]:
sentimentdata['Sentiment'].value_counts()

Sentiment
Positive    415
Negative    218
Neutral      99
Name: count, dtype: int64

## **Modeling**

In [ ]:
# Create a copy of the data
df = sentimentdata.copy()

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    # Remove extra spaces and punctuation
    text = text.strip()
    text = ''.join([c for c in text if c.isalnum() or c.isspace()])
    # Tokenization
    tokens = word_tokenize(text.lower())
    # Removing stop words and lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [ ]:
df['Text'] = df['Text'].apply(preprocess_text)

# Feature extraction
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Text'])
y = df['Sentiment']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
  """
  This function creates and plots the confusion matrix for a given set of true and predicted labels.

  Args:
      y_true: A list containing the true labels.
      y_pred: A list containing the predicted labels.
  """
  # Generate the confusion matrix
  cm = confusion_matrix(y_true, y_pred)

  # Create a subplot for the confusion matrix
  ax = plt.subplot()

  # Plot the confusion matrix using seaborn
  sns.heatmap(cm, annot=True, ax=ax)  # Annotate cells with values

  # Set labels and title
  ax.set_xlabel('Predicted labels')
  ax.set_ylabel('True labels')
  ax.set_title('Confusion Matrix')

  # Set tick labels for x and y axes
  ax.xaxis.set_ticklabels(['Negative', 'Neutral', 'Positive'])
  ax.yaxis.set_ticklabels(['Negative', 'Neutral', 'Positive'])

  # Display the plot
  plt.show()


### **Naive Bayes classifier**

In [25]:
from sklearn.naive_bayes import MultinomialNB
# Training the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Predicting on the test set
y_pred_nb = nb_classifier.predict(X_test)

# Model evaluation
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Accuracy:", accuracy_nb)
print("Classification Report:")
print(classification_report(y_test, y_pred_nb))

plot_confusion_matrix(y_test, y_pred_nb)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Accuracy: 0.8027210884353742
Classification Report:
              precision    recall  f1-score   support

    Negative       0.89      0.84      0.86        49
     Neutral       0.41      0.47      0.44        19
    Positive       0.86      0.86      0.86        79

    accuracy                           0.80       147
   macro avg       0.72      0.72      0.72       147
weighted avg       0.81      0.80      0.81       147



### **Logistic Regression**


In [ ]:
from sklearn.linear_model import LogisticRegression

# Training the Logistic Regression classifier
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train, y_train)

# Predicting on the test set
y_pred_lr = lr_classifier.predict(X_test)

# Model evaluation
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy_lr)
print("Classification Report:")
print(classification_report(y_test, y_pred_lr))

plot_confusion_matrix(y_test, y_pred_lr)

### **Support Vector Machine**

In [ ]:
from sklearn.svm import SVC
# Training the Support Vector Machine classifier
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)

# Predicting on the test set
y_pred_svm = svm_classifier.predict(X_test)

# Model evaluation
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Accuracy:", accuracy_svm)
print("Classification Report:")
print(classification_report(y_test, y_pred_svm))

plot_confusion_matrix(y_test, y_pred_svm)

### **Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Training the Random Forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

# Predicting on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Model evaluation
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy_rf)
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))

plot_confusion_matrix(y_test, y_pred_rf)

### **Recurrent Neural Networks (RNNs)**

In [ ]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
df['Text'] = df['Text'].apply(preprocess_text)

# Create the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Text'])

# Convert the text data to sequences
sequences = tokenizer.texts_to_sequences(df['Text'])

# Pad the sequences
padded_sequences = pad_sequences(sequences, maxlen=100)

# Create the RNN model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=100))
model.add(LSTM(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, df['Sentiment'], epochs=10, batch_size=32)

# Evaluate the model
y_pred_rnn = model.predict(padded_sequences)
y_pred_rnn = np.round(y_pred_rnn)
accuracy_rnn = accuracy_score(df['Sentiment'], y_pred_rnn)
print("Accuracy:", accuracy_rnn)

print("Classification Report:")
print(classification_report(df['Sentiment'], y_pred_rnn))

plot_confusion_matrix(df['Sentiment'], y_pred_rnn)

### **Comparison of Models**

In [ ]:
# Comparing the accuracy of different models
models = ['Naive Bayes', 'Logistic Regression', 'SVM', 'Random Forest', 'RNN']
accuracies = [accuracy_nb, accuracy_lr, accuracy_svm, accuracy_rf, accuracy_rnn]

for model, accuracy in zip(models, accuracies):
    print(f"{model}: {accuracy}")

In [ ]:
# Make predictions on new text data
new_text = ["This is a great product!", "This is a terrible product!"]
new_sequences = tokenizer.texts_to_sequences(new_text)
new_padded_sequences = pad_sequences(new_sequences, maxlen=100)
new_predictions = model.predict(new_padded_sequences)
new_predictions = np.round(new_predictions)
print("Predictions:", new_predictions)

We are going to create 2 more columns one that is lemmentized and the other that is stemmed

In [ ]:
# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Apply stemming to the comments
sentimentdata['clean_comment_stemmed'] = sentimentdata['clean_comment'].apply(lambda x: [stemmer.stem(word) for word in x.split()])
sentimentdata['clean_comment_stemmed_str'] = sentimentdata['clean_comment_stemmed'].apply(' '.join)



In [ ]:
sentimentdata.head()

,clean_comment,category,clean_comment_stemmed,clean_comment_stemmed_str
0,family mormon have never tried explain them t...,1,"[famili, mormon, have, never, tri, explain, th...",famili mormon have never tri explain them they...
1,buddhism has very much lot compatible with chr...,1,"[buddhism, ha, veri, much, lot, compat, with, ...",buddhism ha veri much lot compat with christia...
2,seriously don say thing first all they won get...,-1,"[serious, don, say, thing, first, all, they, w...",serious don say thing first all they won get i...
3,what you have learned yours and only yours wha...,0,"[what, you, have, learn, your, and, onli, your...",what you have learn your and onli your what yo...
4,for your own benefit you may want read living ...,1,"[for, your, own, benefit, you, may, want, read...",for your own benefit you may want read live bu...


In [ ]:
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()
# Apply lemmatization to the comments
sentimentdata['clean_comment_lemmatized'] = sentimentdata['clean_comment'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x.split()])
sentimentdata['clean_comment_lemmatized_str'] = sentimentdata['clean_comment_lemmatized'].apply(' '.join)



In [ ]:
sentimentdata.head()

,clean_comment,category,clean_comment_stemmed,clean_comment_stemmed_str,clean_comment_lemmatized,clean_comment_lemmatized_str
0,family mormon have never tried explain them t...,1,"[famili, mormon, have, never, tri, explain, th...",famili mormon have never tri explain them they...,"[family, mormon, have, never, tried, explain, ...",family mormon have never tried explain them th...
1,buddhism has very much lot compatible with chr...,1,"[buddhism, ha, veri, much, lot, compat, with, ...",buddhism ha veri much lot compat with christia...,"[buddhism, ha, very, much, lot, compatible, wi...",buddhism ha very much lot compatible with chri...
2,seriously don say thing first all they won get...,-1,"[serious, don, say, thing, first, all, they, w...",serious don say thing first all they won get i...,"[seriously, don, say, thing, first, all, they,...",seriously don say thing first all they won get...
3,what you have learned yours and only yours wha...,0,"[what, you, have, learn, your, and, onli, your...",what you have learn your and onli your what yo...,"[what, you, have, learned, yours, and, only, y...",what you have learned yours and only yours wha...
4,for your own benefit you may want read living ...,1,"[for, your, own, benefit, you, may, want, read...",for your own benefit you may want read live bu...,"[for, your, own, benefit, you, may, want, read...",for your own benefit you may want read living ...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
def train_and_evaluate_logisticR(X, y = sentimentdata['category']):
  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

  # Train a logistic regression model
  logistic_model = LogisticRegression()
  logistic_model.fit(X_train, y_train)
  # Evaluate the logistic regression model
  y_pred_logistic = logistic_model.predict(X_test)
  accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
  print("Logistic Regression Accuracy:", accuracy_logistic)

  # Get the confusion matrix
  cm = confusion_matrix(y_test, y_pred_logistic)

  # Create a heatmap of the confusion matrix
  sns.heatmap(cm, annot=True, fmt="d")
  plt.xlabel("Predicted")
  plt.ylabel("Actual")
  plt.title("Confusion Matrix")
  plt.show()



In [ ]:
train_and_evaluate_logisticR(X = sentimentdata['clean_comment_stemmed_str'])

ValueError: could not convert string to float: 'and mota bhai did yoga it grave'

In [ ]:
# Train an SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train)

In [ ]:
# Evaluate the SVM model
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)

In [ ]:
# Train a simple neural network model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000)
nn_model.fit(X_train, y_train)

In [ ]:
# Evaluate the simple neural network model
y_pred_nn = nn_model.predict(X_test)
accuracy_nn = accuracy_score(y_test, y_pred_nn)
print("Simple Neural Network Accuracy:", accuracy_nn)